In [1]:
# DELIVERABLE 2
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Longyearbyen,SJ,78.2186,15.6401,19.24,79,20,3.44,few clouds
1,1,Bambous Virieux,MU,-20.3428,57.7575,75.22,79,15,10.89,few clouds
2,2,Huainan,CN,32.6264,116.9969,56.39,64,97,5.01,light rain
3,3,Puerto Ayora,EC,-0.7393,-90.3518,76.96,79,92,9.95,overcast clouds
4,4,Doka,SD,13.5167,35.7667,85.19,35,55,3.65,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 60


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Huainan,CN,32.6264,116.9969,56.39,64,97,5.01,light rain
6,6,Hermanus,ZA,-34.4187,19.2345,59.90,69,68,9.44,light rain
7,7,Busselton,AU,-33.6500,115.3333,54.09,83,17,6.91,few clouds
9,9,Bethel,US,41.3712,-73.4140,57.67,61,90,6.91,overcast clouds
16,16,Salym,RU,60.0625,71.4789,45.16,74,100,17.34,overcast clouds
19,19,Lebu,CL,-37.6167,-73.6500,55.13,64,82,7.25,broken clouds
26,26,Bluff,NZ,-46.6000,168.3333,52.70,86,100,3.31,overcast clouds
34,34,Fernie,CA,49.5000,-115.0687,47.61,99,100,6.35,overcast clouds
35,35,Fortuna,US,40.5982,-124.1573,55.98,86,75,7.00,broken clouds
36,36,Bellevue,US,47.6104,-122.2007,53.19,89,90,1.01,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                170
City                   170
Country                169
Lat                    170
Lng                    170
Max Temp               170
Humidity               170
Cloudiness             170
Wind Speed             170
Current Description    170
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                169
City                   169
Country                169
Lat                    169
Lng                    169
Max Temp               169
Humidity               169
Cloudiness             169
Wind Speed             169
Current Description    169
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Huainan,CN,56.39,light rain,32.6264,116.9969,
6,Hermanus,ZA,59.90,light rain,-34.4187,19.2345,
7,Busselton,AU,54.09,few clouds,-33.6500,115.3333,
9,Bethel,US,57.67,overcast clouds,41.3712,-73.4140,
16,Salym,RU,45.16,overcast clouds,60.0625,71.4789,
19,Lebu,CL,55.13,broken clouds,-37.6167,-73.6500,
26,Bluff,NZ,52.70,overcast clouds,-46.6000,168.3333,
34,Fernie,CA,47.61,overcast clouds,49.5000,-115.0687,
35,Fortuna,US,55.98,broken clouds,40.5982,-124.1573,
36,Bellevue,US,53.19,overcast clouds,47.6104,-122.2007,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!="")]
clean_hotel_df["Hotel Name"].value_counts()

Toba Guest House                           1
Complejo San Fructuoso                     1
Parsippany Inn and Suites Morris Plains    1
GreenTree Hotel Rongcheng Bus Station      1
The Old Smokery                            1
                                          ..
Genc Ogretmenevi                           1
Kingsgate Hotel Te Anau                    1
Hôtel All Suites Le Teich                  1
Gulisitan Hotel                            1
Комната для намаза                         1
Name: Hotel Name, Length: 152, dtype: int64

In [26]:
# Confirm that hotel_clean_df looks like it should.
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Huainan,CN,56.39,light rain,32.6264,116.9969,GME Huainan Tianjiaan District Chaoyang East R...
6,Hermanus,ZA,59.90,light rain,-34.4187,19.2345,Misty Waves Boutique Hotel
7,Busselton,AU,54.09,few clouds,-33.6500,115.3333,Observatory Guest House
9,Bethel,US,57.67,overcast clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
16,Salym,RU,45.16,overcast clouds,60.0625,71.4789,Gostinitsa Salym


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F </dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [36]:
# set up fig
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))